In [ ]:
import os.path
import math
import rmgpy
import rmgpy.constants as constants
from rmgpy.data.rmg import RMGDatabase
from rmgpy.thermo.nasa import NASA, NASAPolynomial
from rmgpy.species import Species

In [ ]:
databasePath = rmgpy.settings['database.directory']

database = RMGDatabase()
database.load(
    path = databasePath,
    thermoLibraries = [],
    reactionLibraries = [],
    seedMechanisms = [],
    kineticsFamilies = 'none'
    )

In [ ]:
species = Species().fromAdjacencyList("""
1  C u0 p0 c0 {2,S} {5,S} {6,S} {12,S}
2  C u0 p0 c0 {1,S} {3,B} {4,B}
3  C u0 p0 c0 {2,B} {8,S} {9,B}
4  C u0 p0 c0 {2,B} {7,S} {10,B}
5  C u0 p0 c0 {1,S} {7,D} {13,S}
6  C u0 p0 c0 {1,S} {8,D} {14,S}
7  C u0 p0 c0 {4,S} {5,D} {18,S}
8  C u0 p0 c0 {3,S} {6,D} {19,S}
9  C u0 p0 c0 {3,B} {11,B} {15,S}
10 C u0 p0 c0 {4,B} {11,B} {17,S}
11 C u0 p0 c0 {9,B} {10,B} {16,S}
12 H u0 p0 c0 {1,S}
13 H u0 p0 c0 {5,S}
14 H u0 p0 c0 {6,S}
15 H u0 p0 c0 {9,S}
16 H u0 p0 c0 {11,S}
17 H u0 p0 c0 {10,S}
18 H u0 p0 c0 {7,S}
19 H u0 p0 c0 {8,S}
""")

species.thermo = NASA(
    polynomials = [
        NASAPolynomial(
            coeffs = [3.85367, 0.00828265, 0.000234702, -4.14957e-07, 2.24313e-10, 43623.5, 11.5197],
            Tmin = (10, 'K'),
            Tmax = (586.277, 'K'),
        ),
        NASAPolynomial(
            coeffs = [-3.54321, 0.0918488, -6.37889e-05, 2.07596e-08, -2.54734e-12, 43922, 38.4044],
            Tmin = (586.277, 'K'),
            Tmax = (3000, 'K'),
        ),
    ],
    Tmin = (10, 'K'),
    Tmax = (3000, 'K'),
    E0 = (362.64, 'kJ/mol'),
    Cp0 = (33.2579, 'J/(mol*K)'),
    CpInf = (457.296, 'J/(mol*K)'),
)

In [ ]:
thermo = species.thermo.toThermoData()
print thermo
thermo = subtractThermoData(thermo, database.thermo.groups['group'].entries['Cds-CdsCbH'].data)
thermo = addThermoData(thermo, database.thermo.groups['group'].entries['Cds-CdsCbCs'].data)
thermo = addThermoData(thermo, database.thermo.groups['group'].entries['Cs-(Cds-Cds)HHH'].data)
print thermo
thermo.S298.value_si -= constants.R * math.log(3)
print thermo

In [ ]:
def addThermoData(thermoData1, thermoData2):
    """
    Add the thermodynamic data `thermoData2` to the data `thermoData1`,
    and return `thermoData1`.
    """
    for i in range(thermoData1.Tdata.value_si.shape[0]):
        thermoData1.Cpdata.value_si[i] += thermoData2.Cpdata.value_si[i]
    thermoData1.H298.value_si += thermoData2.H298.value_si
    thermoData1.S298.value_si += thermoData2.S298.value_si

    return thermoData1

def subtractThermoData(thermoData1, thermoData2):
    """
    Subtract the thermodynamic data `thermoData2` from the data `thermoData1`,
    and return `thermoData1`.
    """
    for i in range(thermoData1.Tdata.value_si.shape[0]):
        thermoData1.Cpdata.value_si[i] -= thermoData2.Cpdata.value_si[i]
    thermoData1.H298.value_si -= thermoData2.H298.value_si
    thermoData1.S298.value_si -= thermoData2.S298.value_si

    return thermoData1